In [ ]:
import pandas as pd

df = pd.read_csv('./drive/MyDrive/Colab Notebooks/merged.csv')

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [ ]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [ ]:
print(df.shape)

(1674, 6)


In [ ]:
import torch
corpus_embeddings = torch.load("./drive/MyDrive/Colab Notebooks/sbert_embeddings.pt")

<ipython-input-5-e95b97a742c1>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  corpus_embeddings = torch.load("./drive/MyDrive/Colab Notebooks/sbert_embeddings.pt")


In [ ]:
print(corpus_embeddings.shape)

torch.Size([1674, 768])


In [ ]:
class NeuralNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(768, 384),
        nn.ReLU(),
        nn.Linear(384, 192),
        nn.ReLU(),
        nn.Linear(192, 96),
        nn.ReLU(),
        nn.Linear(96, 48),
        nn.ReLU(),
        nn.Linear(48, 24),
        nn.ReLU(),
        nn.Linear(24, 12),
        nn.ReLU(),
        nn.Linear(12, 6),
        nn.ReLU(),
        nn.Linear(6, 3)
    )

  def forward(self, x):
    logits = self.linear_relu_stack(x)
    return logits


In [ ]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=768, out_features=384, bias=True)
    (1): ReLU()
    (2): Linear(in_features=384, out_features=192, bias=True)
    (3): ReLU()
    (4): Linear(in_features=192, out_features=96, bias=True)
    (5): ReLU()
    (6): Linear(in_features=96, out_features=48, bias=True)
    (7): ReLU()
    (8): Linear(in_features=48, out_features=24, bias=True)
    (9): ReLU()
    (10): Linear(in_features=24, out_features=12, bias=True)
    (11): ReLU()
    (12): Linear(in_features=12, out_features=6, bias=True)
    (13): ReLU()
    (14): Linear(in_features=6, out_features=3, bias=True)
  )
)


In [ ]:
import torch.optim as optim

loss_fn = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
X = corpus_embeddings
y = df[['r', 'g', 'b']]

In [ ]:
import tqdm
import numpy as np
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, shuffle=True)
X_train = torch.tensor(X_train, dtype=torch.float32).to(device)
y_train = torch.tensor(y_train.values, dtype=torch.float32).to(device)
X_test = torch.tensor(X_test, dtype=torch.float32).to(device)
y_test = torch.tensor(y_test.values, dtype=torch.float32).to(device)

<ipython-input-10-c0b1895cfa5c>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train = torch.tensor(X_train, dtype=torch.float32).to(device)
<ipython-input-10-c0b1895cfa5c>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test = torch.tensor(X_test, dtype=torch.float32).to(device)


In [ ]:
import copy

In [ ]:
n_epochs = 10000
batch_size = 10
batch_start = torch.arange(0, len(X_train), batch_size)

best_mse = np.inf
best_weights = None
history = []

for epoch in range(n_epochs):
  model.train()
  with tqdm.tqdm(batch_start, unit="batch", mininterval=0, disable=True) as bar:
    bar.set_description(f"Epoch {epoch}")
    for start in bar:
      X_batch = X_train[start:start+batch_size]
      y_batch = y_train[start:start+batch_size]
      y_pred = model(X_batch)
      loss = loss_fn(y_pred, y_batch)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      bar.set_postfix(mse=float(loss))
    model.eval()
    y_pred = model(X_test)
    mse = loss_fn(y_pred, y_test)
    mse = float(mse)
    history.append(mse)
    if mse < best_mse:
      best_mse = mse
      best_weights = copy.deepcopy(model.state_dict())
model.load_state_dict(best_weights)

<All keys matched successfully>

In [ ]:
print(best_mse)

6964.0302734375


In [ ]:
torch.save(model.state_dict(), './model_noscaler.pth')

In [ ]:
model = torch.load('./model_noscaler.pth')

<ipython-input-30-05e0f805750e>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load('./model_noscaler.pth')


In [ ]:
from sentence_transformers import SentenceTransformer, util
import numpy as np

embedder = SentenceTransformer('jhgan/ko-sroberta-multitask')

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.86k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/585 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
query = "바람이 쌩쌩 부는 겨울"
model = NeuralNetwork().to(device) # Create an instance of your model
model.load_state_dict(torch.load('./model_noscaler.pth')) # Load the saved state dictionary
model.eval() # Set the model to evaluation mode
X_test = X_test.to(device)
y_test = y_test.to(device)
y_pred = model(X_test)
mse = loss_fn(y_pred, y_test)
mse = float(mse)
print(mse)

6964.0302734375


<ipython-input-35-3b942cc939f5>:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('./model_noscaler.pth')) # Load the saved state dictionary


In [ ]:
emb_query = torch.tensor(embedder.encode(query))
emb_query = emb_query.to(device)
pred = model(emb_query)
print(pred)

tensor([49.9522, 44.5922, 44.5290], device='cuda:0', grad_fn=<ViewBackward0>)


In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train = X_train.cpu()
X_test = X_test.cpu()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

X_train_scaled = torch.tensor(X_train_scaled, dtype=torch.float32).to(device)
X_test_scaled = torch.tensor(X_test_scaled, dtype=torch.float32).to(device)

In [ ]:
n_epochs = 100
batch_size = 10
batch_start = torch.arange(0, len(X_train), batch_size)

best_mse = np.inf
best_weights = None
history = []

for epoch in range(n_epochs):
  model.train()
  with tqdm.tqdm(batch_start, unit="batch", mininterval=0, disable=True) as bar:
    bar.set_description(f"Epoch {epoch}")
    for start in bar:
      X_batch = X_train_scaled[start:start+batch_size]
      y_batch = y_train[start:start+batch_size]
      y_pred = model(X_batch)
      loss = loss_fn(y_pred, y_batch)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      bar.set_postfix(mse=float(loss))
    model.eval()
    y_pred = model(X_test_scaled)
    mse = loss_fn(y_pred, y_test)
    mse = float(mse)
    history.append(mse)
    if mse < best_mse:
      best_mse = mse
      best_weights = copy.deepcopy(model.state_dict())
model.load_state_dict(best_weights)

<All keys matched successfully>

In [ ]:
# by minmaxscaler
print(best_mse)

20762.7421875


In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = X_train.cpu()
X_test = X_test.cpu()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

X_train_scaled = torch.tensor(X_train_scaled, dtype=torch.float32).to(device)
X_test_scaled = torch.tensor(X_test_scaled, dtype=torch.float32).to(device)

In [ ]:
n_epochs = 100
batch_size = 10
batch_start = torch.arange(0, len(X_train), batch_size)

best_mse = np.inf
best_weights = None
history = []

for epoch in range(n_epochs):
  model.train()
  with tqdm.tqdm(batch_start, unit="batch", mininterval=0, disable=True) as bar:
    bar.set_description(f"Epoch {epoch}")
    for start in bar:
      X_batch = X_train_scaled[start:start+batch_size]
      y_batch = y_train[start:start+batch_size]
      y_pred = model(X_batch)
      loss = loss_fn(y_pred, y_batch)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      bar.set_postfix(mse=float(loss))
    model.eval()
    y_pred = model(X_test_scaled)
    mse = loss_fn(y_pred, y_test)
    mse = float(mse)
    history.append(mse)
    if mse < best_mse:
      best_mse = mse
      best_weights = copy.deepcopy(model.state_dict())
model.load_state_dict(best_weights)

<All keys matched successfully>

In [ ]:
# with standardscaler
print(best_mse)

14839.69140625
